<a href="https://colab.research.google.com/github/michelmalki/MichelMalki/blob/main/CCCS_680_Assignment_3_Question1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pyspark Setup and CSV Input

The following notebook helps to setup pyspark on Colab as shows you how to read and extract contents from a CSV file.

First we download and install Java, Spark and findspark

In [1]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.5.1/spark-3.5.1-bin-hadoop3.tgz
!tar xf spark-3.5.1-bin-hadoop3.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

import os
import sys

import findspark
findspark.init()
findspark.find()

import pyspark

from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F

spark= SparkSession \
       .builder \
       .appName("CCCS 680 Assignment 3 Question 1 ") \
       .getOrCreate()

spark

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Ign:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,550 kB]
Get:14 http://security.ubuntu.com/ubuntu j

Now we map our google drive. Note that you will be prompted for your google account credentials and be provided with an authorization code to insert. The contents of your google drive storage will be mounted at `/content/drive`.

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
purchases_df = spark.read.csv('/content/drive/My Drive/CCCS-680 Assignment 3/purchases.txt.gz', inferSchema=True, header=False, sep='\t')

In [4]:
purchases_df.show()

+----------+-------------------+--------------+--------------------+------+----------+
|       _c0|                _c1|           _c2|                 _c3|   _c4|       _c5|
+----------+-------------------+--------------+--------------------+------+----------+
|2012-01-01|2024-07-25 09:00:00|      San Jose|      Men's Clothing|214.05|      Amex|
|2012-01-01|2024-07-25 09:00:00|    Fort Worth|    Women's Clothing|153.57|      Visa|
|2012-01-01|2024-07-25 09:00:00|     San Diego|               Music| 66.08|      Cash|
|2012-01-01|2024-07-25 09:00:00|    Pittsburgh|        Pet Supplies|493.51|  Discover|
|2012-01-01|2024-07-25 09:00:00|         Omaha| Children's Clothing|235.63|MasterCard|
|2012-01-01|2024-07-25 09:00:00|      Stockton|      Men's Clothing|247.18|MasterCard|
|2012-01-01|2024-07-25 09:00:00|        Austin|             Cameras| 379.6|      Visa|
|2012-01-01|2024-07-25 09:00:00|      New York|Consumer Electronics| 296.8|      Cash|
|2012-01-01|2024-07-25 09:00:00|Corpus Chri

## 1.A

What was the total amount sold

In [5]:
purchases_df.select(F.sum(purchases_df._c4)).show()

+--------------------+
|            sum(_c4)|
+--------------------+
|1.0344579532600112E9|
+--------------------+



## 1.B
What is the total number of transactions per store location?


In [6]:
transactions = purchases_df.groupBy(purchases_df._c2).count().show()

+---------------+-----+
|            _c2|count|
+---------------+-----+
|North Las Vegas|40013|
|        Phoenix|40333|
|          Omaha|40209|
|      Anchorage|39806|
|        Anaheim|40086|
|     Greensboro|40232|
|         Dallas|40368|
|        Oakland|39728|
|         Laredo|40342|
|     Scottsdale|40173|
|    San Antonio|40197|
|    Bakersfield|40326|
|        Raleigh|40261|
|    Chula Vista|40080|
|   Philadelphia|40748|
|     Louisville|40099|
|    Los Angeles|40254|
|       Chandler|39826|
|     Sacramento|40561|
|   Indianapolis|40321|
+---------------+-----+
only showing top 20 rows



# 1.C

What is the total sold for every combination of product type and store location


In [7]:
result = purchases_df.groupBy("_c3", "_c2").agg(F.sum("_c4").alias("total_sold"))
result.show()

+-------------------+------------+-----------------+
|                _c3|         _c2|       total_sold|
+-------------------+------------+-----------------+
|  Health and Beauty|   Charlotte|547280.5699999997|
|             Crafts|      Durham|553803.5200000005|
|   Women's Clothing|        Mesa|572868.2699999996|
|            Cameras|       Plano|548115.3300000005|
|Children's Clothing| Los Angeles|574671.8499999995|
|       Pet Supplies|  Fort Wayne|560785.3400000004|
|                CDs|Indianapolis|572676.8100000003|
|     Men's Clothing| Baton Rouge|563814.1699999993|
|   Women's Clothing|      Aurora|551525.0699999997|
|  Health and Beauty|      Aurora|552101.0300000011|
|     Sporting Goods|     Fremont| 544994.409999999|
|             Crafts| Albuquerque|543748.6199999994|
|            Cameras| Bakersfield|547924.4799999984|
|            Cameras|     El Paso|        581664.08|
|          Computers|Indianapolis|560813.3699999992|
|                CDs|     Lubbock|521722.75999

# 1.D
What is the maximum amount paid at each separate store location

In [8]:
# I do find maximum amount paid in two different ways
result = purchases_df.groupBy("_c2").agg(F.max("_c4").alias("max_amount"))
result.show()

+---------------+----------+
|            _c2|max_amount|
+---------------+----------+
|North Las Vegas|    499.98|
|        Phoenix|    499.99|
|          Omaha|    499.99|
|      Anchorage|    499.99|
|        Anaheim|    499.98|
|     Greensboro|    499.99|
|         Dallas|    499.99|
|        Oakland|    499.99|
|         Laredo|    499.96|
|     Scottsdale|    499.98|
|    San Antonio|    499.98|
|    Bakersfield|    499.97|
|        Raleigh|    499.99|
|    Chula Vista|    499.99|
|   Philadelphia|    499.98|
|     Louisville|    499.98|
|    Los Angeles|    499.99|
|       Chandler|    499.98|
|     Sacramento|    499.96|
|   Indianapolis|    499.98|
+---------------+----------+
only showing top 20 rows



In [9]:
transactions = purchases_df.groupBy(purchases_df._c2).max().show()

+---------------+--------+
|            _c2|max(_c4)|
+---------------+--------+
|North Las Vegas|  499.98|
|        Phoenix|  499.99|
|          Omaha|  499.99|
|      Anchorage|  499.99|
|        Anaheim|  499.98|
|     Greensboro|  499.99|
|         Dallas|  499.99|
|        Oakland|  499.99|
|         Laredo|  499.96|
|     Scottsdale|  499.98|
|    San Antonio|  499.98|
|    Bakersfield|  499.97|
|        Raleigh|  499.99|
|    Chula Vista|  499.99|
|   Philadelphia|  499.98|
|     Louisville|  499.98|
|    Los Angeles|  499.99|
|       Chandler|  499.98|
|     Sacramento|  499.96|
|   Indianapolis|  499.98|
+---------------+--------+
only showing top 20 rows

